In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean            SD         Naive SE       MCSE         ESS    
         lp__  927.803010000  3.2472660017 0.05134378367 0.1269381586  654.412133
accept_stat__    0.938293051  0.0866155403 0.00136951194 0.0019491625 1000.000000
   stepsize__    0.047156325  0.0037622136 0.00005948582 0.0006023610   39.009752
  treedepth__    5.952250000  0.4500228524 0.00711548606 0.0190494885  558.087760
 n_leapfrog__   76.986000000 36.9217091581 0.58378348023 1.7420268478  449.214644
  divergent__    0.000000000  0.0000000000 0.00000000000 0.0000000000         NaN
     energy__ -921.196012000  4.2274348573 0.06684161405 0.1601593448  696.705568
            a    1.074409385  0.7332583058 0.01159383180 0.0217160312 1000.000000
           bp    0.263929255  0.0801222553 0.00126684409 0.0024449544 1000.000000
  a_society.1   -0.201289298  0.2391208841 0.00378083315 0.00564

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*